# San Francisco Rental Prices Dashboard

In this notebook, you will compile the visualizations from the previous analysis into functions that can be used for a Panel dashboard.

In [100]:
# imports
import panel as pn
pn.extension('plotly')
import plotly.express as px
import pandas as pd
import hvplot.pandas
import matplotlib.pyplot as plt
import os
from pathlib import Path
from dotenv import load_dotenv

In [101]:
# Read the Mapbox API key
load_dotenv()
mapbox_token = os.getenv("MAPBOX_API_KEY")

# Set the Mapbox API
px.set_mapbox_access_token(mapbox_token)

# Import Data

In [102]:
# Import the CSVs to Pandas DataFrames
file_path = Path("Data/sfo_neighborhoods_census_data.csv")
sfo_data = pd.read_csv(file_path, index_col="year")

file_path = Path("Data/neighborhoods_coordinates.csv")
df_neighborhood_locations = pd.read_csv(file_path)

- - -

## Panel Visualizations

In this section, you will copy the code for each plot type from your analysis notebook and place it into separate functions that Panel can use to create panes for the dashboard. 

These functions will convert the plot object to a Panel pane.

Be sure to include any DataFrame transformation/manipulation code required along with the plotting code.

Return a Panel pane object from each function that can be used to build the dashboard.

Note: Remove any `.show()` lines from the code. We want to return the plots instead of showing them. The Panel dashboard will then display the plots.

In [113]:
# Define Panel Visualization Functions
def housing_units_per_year():
    """Housing Units Per Year."""
    housing_units_per_year = sfo_data["housing_units"].groupby([sfo_data.index]).mean()  
    return housing_units_per_year.hvplot.bar(
        title="Housing Units in San Francisco from 2010 to 2016",
        xlabel="Year",
        ylabel="Housing Units",
        yformatter="%.0f",
        ylim=(365000, 385000)
    )
           
def average_gross_rent():
    """Average Gross Rent in San Francisco Per Year."""
    average_gross_rent = sfo_data["gross_rent"].groupby([sfo_data.index]).mean()  
    return average_gross_rent.hvplot(
        title="Average Gross Rent in San Francisco",
        xlabel="Year",
        ylabel="Gross Rent"
    )
           
def average_sales_price():
    """Average Sales Prices Per Year."""
    average_sales_price = sfo_data["sale_price_sqr_foot"].groupby([sfo_data.index]).mean()
    return average_sales_price.hvplot(
        title="Average Sale Price per Square Foot in San Francisco",
        xlabel="Year",
        ylabel="Average Sale Price"
    )

def average_price_by_neighborhood():
    """Average Prices by Neighborhood."""
    average_price_by_neighborhood = sfo_data.groupby(["neighborhood", "year"]).mean().reset_index()
    return average_price_by_neighborhood.hvplot(
        x="year",
        y="sale_price_sqr_foot",
        groupby='neighborhood',
        xlabel="Year",
        ylabel="Average Sale Price per Square Foot"
    )

def top_most_expensive_neighborhoods():
    """Top 10 Most Expensive Neighborhoods."""
    neighborhood_data = sfo_data.groupby(['neighborhood']).mean()
    neighborhood_sorted = neighborhood_data.sort_values('sale_price_sqr_foot', ascending=False).reset_index()
    top_most_expensive_neighborhoods = neighborhood_sorted.iloc[:10]
    return top_most_expensive_neighborhoods.hvplot.bar(
        x="neighborhood",
        y="sale_price_sqr_foot",
        rot=30,
        height=450,
        width=600,
        title="Top 10 Expensive Neighborhoods in SF",
        xlabel="Year",
        ylabel="Average Sale Price per Square Foot"
    )

def parallel_coordinates():
    """Parallel Coordinates Plot."""
    neighborhood_data = sfo_data.groupby(["neighborhood"]).mean()
    neighborhood_sorted = neighborhood_data.sort_values("sale_price_sqr_foot", ascending=False).reset_index()
    top_most_expensive_neighborhoods = neighborhood_sorted.iloc[:10]
    return px.parallel_coordinates(top_most_expensive_neighborhoods, color="sale_price_sqr_foot")

def parallel_categories():
    """Parallel Categories Plot."""
    neighborhood_data = sfo_data.groupby(["neighborhood"]).mean()
    neighborhood_sorted = neighborhood_data.sort_values("sale_price_sqr_foot", ascending=False).reset_index()
    top_most_expensive_neighborhoods = neighborhood_sorted.iloc[:10] 
    return px.parallel_categories(
        top_most_expensive_neighborhoods,
        dimensions=["neighborhood", "sale_price_sqr_foot", "housing_units", "gross_rent"],
        color="sale_price_sqr_foot",
        color_continuous_scale=px.colors.sequential.Inferno
    )

def neighborhood_map():
    """Neighborhood Map"""
    neighborhood_prices = sfo_data.groupby(["neighborhood"]).mean().reset_index()
    neighborhood_location_prices = pd.concat([df_neighborhood_locations, neighborhood_prices], axis=1).dropna().reset_index()
    neighborhood_df = neighborhood_location_prices.drop(columns=["neighborhood","index"])
    return px.scatter_mapbox(
        neighborhood_df,
        lat="Lat",
        lon="Lon",
        color="gross_rent",
        hover_name="Neighborhood",
        size="sale_price_sqr_foot",
        zoom=10,
        title="Average Sale Price Per Square Foot and Gross Rent in San Francisco"
    )

## Panel Dashboard

In this section, you will combine all of the plots into a single dashboard view using Panel. Be creative with your dashboard design!

In [114]:
# Create panels to structure the layout of the dashboard
map_1 = pn.Column(
    "#### This dashboard is a visual analysis of historical house units, sale price, and gross rent in San Francisco from 2010 to 2016. You can navigate through the tabs above to explore more.",
    neighborhood_map()
)

housing_units_column = pn.Column(
    housing_units_per_year(),
    average_gross_rent(),
    average_sales_price()
)

neighborhood_summary = pn.panel(
    top_most_expensive_neighborhoods()
)

neighborhood_details = pn.Row(
    average_price_by_neighborhood()
)

row_of_parallel = pn.Row(
    parallel_coordinates(),
    parallel_categories()
)

# Create tabs
visuals = pn.Tabs(
    ("Welcome", map_1),
    ("Yearly Market Analysis", housing_units_column),
    ("Neighborhood Summary", neighborhood_summary),
    ("Neighborhood Analysis", neighborhood_details),
    ("Parallel Plots Analysis", row_of_parallel)
)

panel = pn.Column(
    "##Real Estate Analysis of San Francisco, CA from 2010 to 2016",
    visuals
)

## Serve the Panel Dashboard

In [115]:
panel.servable()

Column
    [0] Markdown(str)
    [1] Tabs
        [0] Column
            [0] Markdown(str)
            [1] Plotly(Figure)
        [1] Column
            [0] HoloViews(Bars)
            [1] HoloViews(Curve)
            [2] HoloViews(Curve)
        [2] HoloViews(Bars)
        [3] Row
            [0] Row
                [0] HoloViews(DynamicMap)
                [1] Column
                    [0] WidgetBox
                        [0] Select(margin=(20, 20, 20, 20), name='neighborhood', options=['Alamo Square', ...], value='Alamo Square', width=250)
                    [1] VSpacer()
        [4] Row
            [0] Plotly(Figure)
            [1] Plotly(Figure)